In [1]:
try:
    import graphene
except ImportError:
    !/opt/conda/envs/python2/bin/pip install graphene

In [2]:
from collections import namedtuple
import json
import omero.clients
from omero.rtypes import unwrap, rlist, rstring
from idr import connection
import graphene

c = connection()
qs = c.getQueryService()

Connected to IDR ...


In [3]:
class AnnotationType(graphene.Enum):
    GENE = 1
    PHENOTYPE = 2


class Annotation(graphene.Interface):
    id = graphene.Int(required=True)
    name = graphene.String(required=True)
    images = graphene.List(graphene.NonNull(lambda: Image))

class AbstractAnnotation(graphene.AbstractType):
    #class Meta:
    #    interfaces = (Annotation,)
    id = graphene.Int(required=True)
    name = graphene.String(required=True)
    images = graphene.List(graphene.NonNull(lambda: Image))

    def resolve_images(self, info):
        p = omero.sys.ParametersI()
        p.addId(self.id)
        rs = qs.projection(
            """
            SELECT ial.parent.id, ial.parent.name
            FROM ImageAnnotationLink ial
            WHERE ial.child.id=:id
            AND ial.child.class=MapAnnotation
            """, p)
        return (Image(*r) for r in unwrap(rs))

class Gene(graphene.ObjectType, AbstractAnnotation):
    class Meta:
        interfaces = (Annotation,)
    ensemblid = graphene.String(required=True)

    #qs.projection("SELECT ann.id,ann.class,mv.name,mv.value,ann.ns FROM ImageAnnotationLink ial JOIN ial.child as ann JOIN ann.mapValue as mv WHERE parent=1030631 AND ann.class=MapAnnotation",None) 
    #def resolve_images



class Phenotype(graphene.ObjectType, AbstractAnnotation):
    class Meta:
        interfaces = (Annotation,)
    cmpoterm = graphene.String(required=True)


class Image(graphene.ObjectType):
    id = graphene.Int(required=True)
    name = graphene.String(required=True)
    annotations = graphene.List(
        graphene.NonNull(lambda: Annotation), mapr=graphene.String())

    def resolve_annotations(self, info, mapr=None):
        p = omero.sys.ParametersI()
        p.addId(self.id)
        NsDetails = namedtuple('NsDetails', ['short', 'cls', 'pk'])
        nsmap = {
            'openmicroscopy.org/mapr/gene': NsDetails(
                'ge', Gene, 'Gene Symbol'),
            'openmicroscopy.org/mapr/phenotype': NsDetails(
                'ph', Phenotype, 'Phenotype'),
        }
        if mapr:
            k = 'openmicroscopy.org/mapr/' + mapr
            v = nsmap[k]
            nsmap = {k: v}
        query = []
        for ns, nsd in nsmap.items():
            t = nsd.short
            p.addString(t + 'ns', ns)
            p.addString(t + 'pk', nsd.pk)
            query.append('(ial.child.ns=:{} AND mv.name=:{})'.format(
                t + 'ns', t + 'pk'))
        q = """
            SELECT ial.child.id, mv.value, ial.child.ns
            FROM ImageAnnotationLink ial
            JOIN ial.child.mapValue AS mv
            WHERE ial.parent.id=:id
            AND ial.child.class=MapAnnotation
            AND ({})
            """.format(' OR '.join(query))
        #print(q)
        rs = qs.projection(q, p)
        return (nsmap[r[-1]].cls(*r[:-1]) for r in unwrap(rs))


class Query(graphene.ObjectType):
    image = graphene.Field(Image, id=graphene.Int())
    geneSymbols = graphene.List(Gene, symbol=graphene.String())
    
    def resolve_image(self, info, id):
        p = omero.sys.ParametersI()
        p.addId(id)
        rs = qs.projection("""
            SELECT id, name
            FROM Image
            WHERE id=:id
            """, p)
        if rs:
            return Image(*unwrap(rs[0]))

    def resolve_geneSymbols(self, info, symbol):
        p = omero.sys.ParametersI()
        p.page(0, 10)
        p.addString('symbol', symbol)
        rs = qs.projection("""
            SELECT ann.id, mv.value
            FROM MapAnnotation ann
            JOIN ann.mapValue AS mv
            WHERE ann.ns='openmicroscopy.org/mapr/gene'
            AND mv.name='Gene Symbol'
            AND lower(mv.value)=:symbol
            """, p)
        return (Gene(*r) for r in unwrap(rs))



schema = graphene.Schema(query=Query, types=(Gene, Phenotype, AbstractAnnotation))
#print(schema.execute('{geneSymbols(symbol: "polo") {id}}').data)
print(schema.execute('{image(id: 1030631) {annotations {name}}}').data)

OrderedDict([('image', OrderedDict([('annotations', [OrderedDict([('name', 'defective cell-cell aggregation')]), OrderedDict([('name', 'polo')])])]))])


In [4]:
# TODO
from promise import Promise
from promise.dataloader import DataLoader

class ImageLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        return Promise.resolve([get_user(id=key) for key in keys])


def qlbuild(*args):
    q = ' '.join(qlbuild(*arg) if isinstance(arg, list) else arg
                 for arg in args)
    return '{' + q + '}'


def execprint(*args):
    if len(args) > 1:
        q = qlbuild(*args)
    else:
        q = args[0]
    print('query: ' + q)
    s = schema.execute(q).data
    print(json.dumps(s, indent=2))

#print(schema.execute('{geneSymbols(symbol: "polo") {id}}').data)
#print(schema.execute('{ geneSymbols(symbol: "polo") {id images {id}}}').data)
execprint('image(id: 1030631)', ['id'])
execprint('image(id: 1030631)', ['id', 'name', 'annotations',
                                 ['__typename', 'id', 'name']])

# execprint('image(id: 1030631)', ['id', 'name', 'annotations',
#     ['__typename', 'id', 'name', 'images', ['id', 'name']]])

query: {image(id: 1030631) {id}}
{
  "image": {
    "id": 1030631
  }
}
query: {image(id: 1030631) {id name annotations {__typename id name}}}
{
  "image": {
    "id": 1030631, 
    "name": "Primary_062 [Well B11 Field #1]", 
    "annotations": [
      {
        "__typename": "Phenotype", 
        "id": 6672118, 
        "name": "defective cell-cell aggregation"
      }, 
      {
        "__typename": "Gene", 
        "id": 6672872, 
        "name": "polo"
      }
    ]
  }
}


In [5]:
execprint('geneSymbols(symbol: "polo")', ['id', 'name', 'images',
    ['id', 'name', 'annotations(mapr: "phenotype")', ['__typename', 'id', 'name']]])


query: {geneSymbols(symbol: "polo") {id name images {id name annotations(mapr: "phenotype") {__typename id name}}}}
{
  "geneSymbols": [
    {
      "id": 6672872, 
      "name": "polo", 
      "images": [
        {
          "id": 1030631, 
          "name": "Primary_062 [Well B11 Field #1]", 
          "annotations": [
            {
              "__typename": "Phenotype", 
              "id": 6672118, 
              "name": "defective cell-cell aggregation"
            }
          ]
        }, 
        {
          "id": 1030633, 
          "name": "Primary_062 [Well B11 Field #3]", 
          "annotations": [
            {
              "__typename": "Phenotype", 
              "id": 6672118, 
              "name": "defective cell-cell aggregation"
            }
          ]
        }, 
        {
          "id": 1030632, 
          "name": "Primary_062 [Well B11 Field #2]", 
          "annotations": [
            {
              "__typename": "Phenotype", 
              "id": 6672118